<a href="https://colab.research.google.com/github/MarsZDF/bfl-hack-sirius/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sirius Demo 🌟

**AI Image Morphing Pipeline using Claude Vision + FLUX.2**

This notebook demonstrates how to use **Sirius** to create smooth, narrative-driven video transitions between images.

## 1. Setup & Installation
Install the Sirius library and its dependencies.

In [ ]:
# Install sirius directly from GitHub
!pip install --upgrade --force-reinstall --no-cache-dir git+https://github.com/MarsZDF/bfl-hack-sirius.git

# Install optional dependencies for this demo
!pip install rich nest-asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()
print("✅ Asyncio nested loops enabled.")

## 2. API Keys
Sirius requires:
1. **Anthropic API Key** (for Claude Vision 'Director')
2. **Runware API Key** (for Image Generation)

In [ ]:
import os
from getpass import getpass

# Helper to set env vars safely
def set_key(name, prompt_text):
    if not os.environ.get(name):
        key = getpass(prompt_text)
        if key:
            os.environ[name] = key
        else:
            print(f"⚠️ {name} not set.")

print("Please enter your API keys (press Enter to skip if already set in environment):")

set_key("ANTHROPIC_API_KEY", "Enter Anthropic API Key (sk-ant-...): ")
set_key("RUNWARE_API_KEY", "Enter Runware API Key: ")

print("\n✅ Setup complete.")

## 3. Prepare Images
We need a **Start Image** and an **End Image**. You can upload your own to the file browser on the left, or run the cell below to download sample assets.

In [ ]:
import requests
from PIL import Image as PILImage
from io import BytesIO
from IPython.display import display, Image as IPyImage

def download_image(url, filename):
    headers = {
        'User-Agent': 'SiriusDemo/1.0 (https://github.com/MarsZDF/bfl-hack-sirius)'
    }
    try:
        print(f"Downloading {url}...")
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        img = PILImage.open(BytesIO(response.content))
        # Convert to RGB if necessary (e.g. RGBA pngs)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(filename)
        return filename
    except Exception as e:
        print(f"❌ Failed to download {url}: {e}")
        # Fallback: Create a placeholder image if download fails
        print("Creating placeholder image...")
        img = PILImage.new('RGB', (800, 600), color = (73, 109, 137))
        img.save(filename)
        return filename

# Create assets folder
os.makedirs("assets", exist_ok=True)

# Sample images (Use original full-res URLs to avoid thumbnail 429s)
url_a = "https://upload.wikimedia.org/wikipedia/commons/4/4d/Cat_November_2010-1a.jpg"
url_b = "https://upload.wikimedia.org/wikipedia/commons/9/90/Labrador_Retriever_portrait.jpg"

image_a = "assets/start.png"
image_b = "assets/end.png"

if not os.path.exists(image_a):
    download_image(url_a, image_a)

if not os.path.exists(image_b):
    download_image(url_b, image_b)

print(f"Start: {image_a}")
print(f"End:   {image_b}")

if os.path.exists(image_a):
    display(IPyImage(filename=image_a, width=300))
if os.path.exists(image_b):
    display(IPyImage(filename=image_b, width=300))

## 4. Plan & Preview (Director Mode)
Before generating the full video, let's ask the Director (Claude) to plan the transition. You can provide additional context or instructions (e.g., 'Make it cybernetic', 'Use warm lighting').

In [ ]:
from sirius import morph, plan_morph, TransitionStyle
from sirius.pipeline import morph

# 0. User Context
user_context = input("Enter any specific instructions for the transition (or press Enter to skip): ")
if not user_context.strip():
    user_context = None

# 1. Plan the transition
print("\nPlanning transition...")
plan = plan_morph(
    image_a,
    image_b,
    frame_count=16, # Smooth 16-frame transition
    transition_style=TransitionStyle.MORPH,
    user_context=user_context
)

# Show the prompts
for i, p in enumerate(plan.prompts):
    print(f"{i+1}: {p[:100]}...")

# 2. Generate Preview (Flux Schnell)
print("\nGenerating preview (fast)... this might take 20-30s")
preview_result = morph(
    image_a,
    image_b,
    plan=plan,
    preview=True, # Uses Schnell model
    output_dir="outputs/preview"
)

print(f"Preview saved: {preview_result.video_path}")
display(IPyImage(filename=preview_result.video_path, width=800))

## 5. Render Final Video
Now we generate the high-quality video using **Flux Pro/Klein** (depending on backend). We'll add 'boomerang' mode to loop it smoothly.

In [ ]:
from IPython.display import Video

def on_progress(update):
    print(f"{update.progress:.0%} - {update.message}")

print("Generating full video... this might take 2-3 minutes")
result = morph(
    image_a,
    image_b,
    plan=plan,     # Reuse the 16-frame plan
    preview=False, # High quality mode
    boomerang=False, # Linear A -> B (Set to True for loop)
    on_progress=on_progress,
    output_dir="outputs/final"
)

print(f"\nVideo saved: {result.video_path}")
display(Video(result.video_path, embed=True, width=600))

## 6. Gallery of Styles
Let's see how other styles would handle this transition. We'll generate small 4-frame previews for Narrative, Fade, and Surreal styles.

In [ ]:
styles = [TransitionStyle.FADE, TransitionStyle.SURREAL, TransitionStyle.NARRATIVE]

for style in styles:
    print(f"\n🎨 Previewing Style: {style.value.upper()}")
    try:
        p_result = morph(
            image_a, 
            image_b,
            frame_count=4, # Just 4 frames for speed
            transition_style=style,
            preview=True, # Fast Flux Schnell
            output_dir=f"outputs/style_{{style.value}}",
            user_context=user_context # Pass the same context
        )
        display(IPyImage(filename=p_result.video_path, width=600))
    except Exception as e:
        print(f"Skipping {style}: {e}")